## Fitting Steady-State Pulse-Labeling Dynamics

##### Import necessary modules

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy as sc
import scipy.optimize as opt
import sympy as sp
from matplotlib.lines import Line2D

##### Define functions to parse data, fit models, and plot results

In [4]:
def parse_sub_pepscanfilt_csv(file_path):
    parsed_timepoint_output = pd.read_csv(file_path, sep=',',usecols=['protein','pep','U/F','L/F','U/L'])
    parsed_timepoint_output = parsed_timepoint_output.sort_values(by=['protein'])
    return parsed_timepoint_output

In [5]:
def protein_median(parsed_csv):
    p_median_1 = parsed_csv.groupby(['protein'])
    p_median_1 = p_median_1['L/F'].agg(np.median)
    
    p_median_2 = parsed_csv.groupby(['protein'])
    p_median_2 = p_median_2['U/F'].agg(np.median)
    
    p_median_3 = parsed_csv.groupby(['protein'])
    p_median_3 = p_median_3['Total'].agg(np.median)
    
    p_median_4 = parsed_csv.groupby(['protein'])
    p_median_4 = p_median_4['Frac_labeled'].agg(np.median)
    
    unique_prots = parsed_csv['protein'].unique()
    
    column_names = ['protein','L/F','U/F', 'Total', 'Frac_labeled']
    protein_median = pd.DataFrame(columns = column_names)
    
    index = []
    for itr in range(len(unique_prots)):
        new_row = {'protein':unique_prots[itr],'L/F':p_median_1[itr],'U/F':p_median_2[itr], 'Total':p_median_3[itr], 'Frac_labeled':p_median_4[itr]}
        protein_median = protein_median.append(new_row, ignore_index = True)
        index.append(itr)

    protein_median.set_index(['protein'], inplace=True)
        
    return protein_median

In [6]:
def compare_timepoints(pmeds_lib, dil_times, dil_factors):
    prot_meds_keys = list(pmeds_lib.keys())
    prot_meds_keys.sort()
    
    joint_df = pmeds_lib[prot_meds_keys[0]].join(pmeds_lib[prot_meds_keys[1]], lsuffix = '_' + str(prot_meds_keys[0]), rsuffix = '_' + str(prot_meds_keys[1]), how = 'inner')
    
    if len(prot_meds_keys) > 2:
        for i in range(2, len(prot_meds_keys)):
            l = "L/F" + "_" + str(prot_meds_keys[i])
            u = "U/F" + "_" + str(prot_meds_keys[i])
            t = "Total" + "_" + str(prot_meds_keys[i])
            f = "Frac_labeled" + "_" + str(prot_meds_keys[i])
            
            joint_df = joint_df.join(pmeds_lib[prot_meds_keys[i]], how = 'inner')
            joint_df.rename(columns = {"L/F": l, "U/F": u, "Total": t, "Frac_labeled": f}, inplace = True)
 
    return prot_meds_keys, joint_df

In [7]:
def pulse_fitting(dirname, OD_data, dil_times, dil_factors, global_k = True,  extension = '_pepscanfilt.csv'):
    '''
    PULSE_FITTING reads a series of .csv files output by pysodist to generate OD-corrected light/heavy and pulse/heavy ratios,
    finds the (pulse/heavy)(pulse/heavy + light/heavy) fractions across the time series, and fits to models 0, 1, and 3 with fixed
    k determined from the OD data, adjusted by the dilution factors used during the course of the experiment
    '''
    
    #updating matplotlib parameters
    plt.rcParams.update({'font.size': 18})
    plt.rcParams.update({'axes.linewidth': 1.0})
    
    #read in data
    parsed_data = {}
    for i in os.listdir(dirname):
        if i.endswith(extension):
            key = int(i.split('_')[0])
            parsed_data[key] = parse_sub_pepscanfilt_csv(dirname + i)

            for j in range(0, len(dil_times)):
                if key > dil_times[j]:
                    parsed_data[key]['U/F'] = parsed_data[key]['U/F']*dil_factors[j]
                    parsed_data[key]['L/F'] = parsed_data[key]['L/F']*dil_factors[j]

            parsed_data[key]['Total'] = parsed_data[key]['U/F'] + parsed_data[key]['L/F']
            parsed_data[key]['Frac_labeled'] = parsed_data[key]['L/F']/(parsed_data[key]['U/F'] + parsed_data[key]['L/F'])

    #calculate median by protein
    prot_meds = {}
    for i in parsed_data.keys():
        prot_meds[i] = protein_median(parsed_data[i])
    
    #compare timepoints
    time, time_df = compare_timepoints(prot_meds, dil_times, dil_factors)
    
    #create dfs with total U+L and fraction labeled L/U+L
    totals = time_df[time_df.columns[time_df.columns.str.contains('Total')]]
    fracs = time_df[time_df.columns[time_df.columns.str.contains('Frac_labeled')]]
    
    fig1, ax1 = plt.subplots(1, 2)
    fig1.set_size_inches(20, 5)
    for i in np.arange(0, len(fracs.index), 1):
        ax1[0].plot(time, fracs.iloc[i, :])
        ax1[0].set_ylim(0, 1)
        ax1[0].set_xlabel('Time (min)')
        ax1[0].set_ylabel('Labeled/(labeled + unlabeled)')
        
        ax1[1].plot(time, totals.iloc[i, :])
        ax1[1].set_xlabel('Time (min)')
        ax1[1].set_ylabel('Labeled + unlabeled')
    
    fig1.savefig(dirname + 'totals_fracs.png')
    
    #adjusting OD data
    for i in range(0, len(dil_times)):
        for j in range(0, len(time)):
            if time[j] > dil_times[i]:
                OD_data[j] = OD_data[j]*dil_factors[i]
    
    #fitting OD data to get global k value
    def single_exp(t, a, k_app, c):
        return a * np.exp(k_app * t) + c
    param_guess = [1, 10**-4, 0]
    OD_fit = opt.curve_fit(single_exp, time, OD_data, p0 = param_guess)[0]
        
    assert type(global_k) == bool
    
    k = OD_fit[1]
    k_list = []
    
    if global_k == False:
        for prot in totals.index:
            totals_data = np.array([totals.at[prot, i] for i in totals.columns])
            totals_fit = opt.curve_fit(single_exp, time, totals_data, p0 = param_guess)[0]
            k_list.append(totals_fit[1])
    
    #plotting OD fit
    fig2, ax2 = plt.subplots(1, 1)
    ax2.plot(time, OD_data, 'xk')
    max_time = np.max(time)
    pred_t = np.linspace(0, max_time, max_time*1000)
    pred_y = single_exp(pred_t, *OD_fit)
    ax2.plot(pred_t, pred_y, '-r')
    ax2.set_xlabel('Time (min)')
    ax2.set_ylabel('Adjusted OD600')
    fig2.savefig(dirname + 'OD_fit.png')
    
    #defining functions for each model based on whether or not k is global
    if global_k == True:
        def model0(t, P):   
            return 1.0 + P*np.exp(-k*(1.0 + 1.0/P)*t) - (1.0 + P)*np.exp(-k*t)
        
        def model1(t, P1, P2):
            return 1.0 - k*(1.0 + P2)/(P2 - P1 - P1*P2)*np.exp(-k*(1.0 + P1 + P1*P2)*t) + (k*(1.0 + P2)/(P2 - P1 - P1*P2) - 1.0)*np.exp(-k*(1.0 + P2)*t)

        def model3(t, P, beta):
            return ((k + P*(k + beta))/(P*(k + beta) - k))*(1.0 - (k + beta)/(k*(2.0 - P) + beta*(1.0 - P))*np.exp(-1.0*(P*(k + beta) - k)*t) + (P*(k + beta) - k)/(k*(2.0 - P) + beta*(1.0 - P))*np.exp(-1.0*(k + beta)*t))
    
        
        def model0_global(combined_x_data, P):

            results = np.array([])

            for i in np.arange(0, len(fracs.index), 1):
                extracted_data = combined_x_data[len(time)*i:len(time)*(i + 1)]
                results = np.append(results, model0(extracted_data, P))

            return results

        
        def model1_global(combined_x_data, P1, P2):

            results = np.array([])

            for i in np.arange(0, len(fracs.index), 1):
                extracted_data = combined_x_data[len(time)*i:len(time)*(i + 1)]
                results = np.append(results, model1(extracted_data, P1, P2))

            return results

        
        def model3_global(combined_x_data, P, *betas):
            results = np.array([])

            betas = np.array([beta for beta in betas])
            for i in np.arange(0, len(betas)):
                extracted_data = combined_x_data[len(time)*i:len(time)*(i +1)]
                results = np.append(results, model3(extracted_data, P, betas[i]))

            return results
        
    elif global_k == False:
        

        def model0_global(combined_x_data, P):

            results = np.array([])

            for i in np.arange(0, len(fracs.index), 1):
                extracted_data = combined_x_data[len(time)*i:len(time)*(i + 1)]
                k = k_list[i]
                def model0(t, P):   
                    return 1.0 + P*np.exp(-k*(1.0 + 1.0/P)*t) - (1.0 + P)*np.exp(-k*t)
                results = np.append(results, model0(extracted_data, P))

            return results

        
        def model1_global(combined_x_data, P1, P2):

            results = np.array([])

            for i in np.arange(0, len(fracs.index), 1):
                extracted_data = combined_x_data[len(time)*i:len(time)*(i + 1)]
                k = k_list[i]
                def model1(t, P1, P2):
                    return 1.0 - k*(1.0 + P2)/(P2 - P1 - P1*P2)*np.exp(-k*(1.0 + P1 + P1*P2)*t) + (k*(1.0 + P2)/(P2 - P1 - P1*P2) - 1.0)*np.exp(-k*(1.0 + P2)*t)
                results = np.append(results, model1(extracted_data, P1, P2))

            return results
        
        def model3_global(combined_x_data, P, *betas):

            results = np.array([])

            betas = np.array([beta for beta in betas])

            for i in np.arange(0, len(betas)):
                extracted_data = combined_x_data[len(time)*i:len(time)*(i +1)]
                k = k_list[i]
                def model3(t, P, beta):
                    return ((k + P*(k + beta))/(P*(k + beta) - k))*(1.0 - (k + beta)/(k*(2.0 - P) + beta*(1.0 - P))*np.exp(-1.0*(P*(k + beta) - k)*t) + (P*(k + beta) - k)/(k*(2.0 - P) + beta*(1.0 - P))*np.exp(-1.0*(k + beta)*t))

                results = np.append(results, model3(extracted_data, P, betas[i]))

            return results
        
                
    def calc_resids(function, x_vals, y_vals, *function_params):   
        resids = [(y_vals[i] - function(x_vals[i], *function_params)) for i in range(0, len(y_vals))] 
        sq_resids = [resids[i]**2 for i in range (0, len(resids))] 
        sum_sq_resids = sum(sq_resids)
        return resids, sum_sq_resids
                
    def calc_resids_global(function, x_vals, y_vals, *function_params): 
        func_data = function(x_vals, *function_params)
        resids = [(y_vals[i] - func_data[i]) for i in range(0, len(y_vals))]
        sq_resids = [resids[i]**2 for i in range (0, len(resids))] 
        sum_sq_resids = sum(sq_resids)
        return resids, sum_sq_resids
    
    fit_results = pd.DataFrame(index = ['model0', 'model1', 'model3'], columns = ['Parameters', 'SSR'])
    
    function_set = [model0_global, model1_global, model3_global]
    
    #updating matplotlib parameter for plotting
    plt.rcParams.update({'font.size': 24})
    plt.rcParams.update({'axes.linewidth': 3.0})
    
    #running the actual fitting with each of the three models
    for function in function_set:
        nrows = int(np.ceil(len(fracs)/3))
        fig3, ax3 = plt.subplots(nrows, 3)    
        ax3 = ax3.flatten()
        fig3.set_size_inches(40, 300)
        style = np.array(['.k', '--r'])
        fig4, ax4 = plt.subplots(1, 1)
        
        if function == model0_global:
            param_guess = 1000
            lb = 0
            ub = np.inf
            paramets = ['P']
            if global_k == True:
                local_function = model0
        elif function == model1_global:
            param_guess = [1000, 1]
            lb = [0, 0]
            ub = [np.inf, np.inf]
            paramets = ['P1', 'P2']
            if global_k == True:
                local_function = model1
        elif function == model3_global:
            beta_guess = np.zeros(len(fracs.index))
            beta_ub = 10**10*np.ones(len(fracs.index))
            beta_lb = np.zeros(len(fracs.index))
            param_guess = np.append([1000], beta_guess)
            lb = np.append([0], beta_lb)
            ub = np.append(np.inf, beta_ub)
            paramets = ['P', 'Beta']
            if global_k == True:
                local_function = model3
        
        #generating the global x and y data arrays
        x_data = np.tile(time, len(fracs.index)) 
        y_data = np.array([])
        for protein in fracs.index:
            
            y_data = np.append(y_data, fracs.loc[protein, :])        
            y_data = np.array(y_data, dtype = np.float64)
        
        #try running the fit, except if there's a RunTime error (unable to fit)
        try:
            fit_results_i = opt.curve_fit(function, x_data, y_data, p0 = param_guess, bounds = (lb, ub))[0]
            SSR = calc_resids_global(function, x_data, y_data, *fit_results_i)[1]

            preds_t = np.linspace(0, max(time), num = max(time)*100) 
        
            for i in range(len(fracs.index)):
                ax3[i].scatter(x_data[i*len(fracs.columns):(i + 1)*len(fracs.columns)], y_data[i*len(fracs.columns):(i + 1)*len(fracs.columns)], s = 40.0, c = 'k')

                if function == model0_global or function == model1_global:
                    local_params = fit_results_i
                elif function ==  model3_global:
                    local_params = fit_results_i[[0, i + 1]]
                
                if global_k == False:
                    k = k_list[i]
                    if function == model0_global:
                        def model0(t, P):   
                            return 1.0 + P*np.exp(-k*(1.0 + 1.0/P)*t) - (1.0 + P)*np.exp(-k*t)
                        local_function = model0
                    if function == model1_global:
                        def model1(t, P1, P2):
                            return 1.0 - k*(1.0 + P2)/(P2 - P1 - P1*P2)*np.exp(-k*(1.0 + P1 + P1*P2)*t) + (k*(1.0 + P2)/(P2 - P1 - P1*P2) - 1.0)*np.exp(-k*(1.0 + P2)*t)
                        local_function = model1
                    if function == model3_global:
                        def model3(t, P, beta):
                            return ((k + P*(k + beta))/(P*(k + beta) - k))*(1.0 - (k + beta)/(k*(2.0 - P) + beta*(1.0 - P))*np.exp(-1.0*(P*(k + beta) - k)*t) + (P*(k + beta) - k)/(k*(2.0 - P) + beta*(1.0 - P))*np.exp(-1.0*(k + beta)*t))
                        local_function = model3
                
                #plot real data vs predicted values from fit
                preds_y = local_function(preds_t, *local_params)
                
                ax3[i].plot(preds_t, preds_y, '--r', linewidth = 6.0)

                ax3[i].set_ylim([0, 1])
                ax3[i].set_xlabel('Time (min)')
                protein_name = fracs.index[i].split('_')[0]
                ax3[i].set_ylabel(', '.join([protein_name, 'L/(U+L)']))


            #clean up the figure
            if np.mod(i + 1, 3) == 1:
                fig3.delaxes(ax3[i + 1])
                fig3.delaxes(ax3[i + 2])
            elif np.mod(i + 1, 3) == 2:
                fig3.delaxes(ax3[i + 1])
            
            #save the figures
            if global_k == True:
                fig3.savefig(dirname + '_'.join([str(function).split('>.')[1].split(' ')[0], 'fits.png']))
                fig4.savefig(dirname + '_'.join([str(function).split('>.')[1].split(' ')[0], 'resids.png']))
            elif global_k == False:
                fig3.savefig(dirname + '_'.join([str(function).split('>.')[1].split(' ')[0], 'fits_non-global-k.png']))
                fig4.savefig(dirname + '_'.join([str(function).split('>.')[1].split(' ')[0], 'resids_non-global-k.png']))
                
        except RuntimeError:
            fit_results_i = np.nan
        
        #store the results of the fit in a df
        model_name = str(function).split('>.')[1].split(' ')[0].split('_')[0]
        fit_results.loc[model_name, 'Parameters'] = fit_results_i
        fit_results.loc[model_name, 'SSR'] = SSR
                
        
    if global_k == True:
        return k, fracs, totals, fit_results
    else:
        return k_list, fracs, totals, fit_results

##### Parsing and fitting data

In [8]:
ods = [0.5, 0.6, 0.7, 0.8, 0.9, 1.1, 1.4, 1.5, 1.8, 2.1, 2.3, 2.4, 3.0, 1.9, 2.3, 2.6, 3.0, 1.8, 2.1, 2.4]
k_fit, fracs_df, totals_df, fits_df = pulse_fitting('./allpepscanfilt/', ods, [360, 480], [2, 2], global_k = True)

FileNotFoundError: [WinError 3] The system cannot find the path specified: './allpepscanfilt/'

In [12]:
k_fit

0.005340380483213396

In [14]:
frcs = [0.500, 0.401, 0.372, 0.39, 0.403, 0.418, 0.421, 0.429, 0.433, 0.437, 0.439, 0.442, 0.443, 0.447, 0.448, 0.449, 0.451, 0.453, 0.454, 0.457]

j = 0
for i in range(0, 600, 30):
    print('pysodist run_isodist ./Gal_15N_' + str(i) + '_DIA/pd_exported_peaks.tsv /nobackup/users/lkinman/software/pysodist/fortran/isodist ./model_files/atoms.txt ./model_files/U_fix' + '{:.3f}'.format(frcs[j]).split('.')[1] + 'N_998N_t' + str(i) + '.txt --threads 160 --pysodist_input ./Gal_15N_' + str(i) + '_DIA/pd_parsed_report.tsv')
    j = j + 1

pysodist run_isodist ./Gal_15N_0_DIA/pd_exported_peaks.tsv /nobackup/users/lkinman/software/pysodist/fortran/isodist ./model_files/atoms.txt ./model_files/U_fix500N_998N_t0.txt --threads 160 --pysodist_input ./Gal_15N_0_DIA/pd_parsed_report.tsv
pysodist run_isodist ./Gal_15N_30_DIA/pd_exported_peaks.tsv /nobackup/users/lkinman/software/pysodist/fortran/isodist ./model_files/atoms.txt ./model_files/U_fix401N_998N_t30.txt --threads 160 --pysodist_input ./Gal_15N_30_DIA/pd_parsed_report.tsv
pysodist run_isodist ./Gal_15N_60_DIA/pd_exported_peaks.tsv /nobackup/users/lkinman/software/pysodist/fortran/isodist ./model_files/atoms.txt ./model_files/U_fix372N_998N_t60.txt --threads 160 --pysodist_input ./Gal_15N_60_DIA/pd_parsed_report.tsv
pysodist run_isodist ./Gal_15N_90_DIA/pd_exported_peaks.tsv /nobackup/users/lkinman/software/pysodist/fortran/isodist ./model_files/atoms.txt ./model_files/U_fix390N_998N_t90.txt --threads 160 --pysodist_input ./Gal_15N_90_DIA/pd_parsed_report.tsv
pysodist ru

'0.500'